## Code for Portland elections


In [ ]:
#For district one
# Note that for the Portland elections, we are using the cleaned pickel file

from votekit.pref_profile import PreferenceProfile
from votekit.cleaning import remove_and_condense

profile = PreferenceProfile.from_pickle('/Users/ss2776/Downloads/Portland_D1_cleaned_votekit_pref_profile-2.pkl')
profile  = remove_and_condense(['Uncertified Write In'], profile)
 

In [ ]:

## for computing σUM score for ranking, change the voting_rule to any other voting rule function defined in Voting_rules.py
## you can use similar syntax for the winner set version 


from Voting_rules import Ranked_Borda, Ranked_Plurality, Ranked_STV

from fairness_metric import sigma_UM

sigma_UM(profile, Ranked_Borda)  # Example using Borda count 


np.float64(0.866881502236788)

In [ ]:

## for computing σIIA score for ranking, change the voting_rule to any other voting rule function defined in Voting_rules.py
## you can use similar syntax for the winner set version 

from Voting_rules import Ranked_Borda, Ranked_Plurality, Ranked_STV

from fairness_metric import sigma_IIA

sigma_IIA(profile, Ranked_Borda)  # Example using Borda count

0.9910714285714286

In [18]:
## Code for Portland elections for District 2,3,4.
## the below code is only for district 2, but you can change the file path to the corresponding csv for district 3 and 4

import pandas as pd

df_2 = pd.read_csv('City_of_Portland__Councilor__District_2_2024_11_29_17_26_12.cvr.csv')



In [19]:
# stores all columns that have ranking information
rank_columns = {i:[col for col in df_2.columns if f'{i}:Number' in col] for i in range(1,7)}
all_rank_cols = [col for col_list in rank_columns.values() for col in col_list]

In [20]:
D2_voters_df = df_2[df_2[all_rank_cols].sum(axis=1) > 0].reset_index(drop=True) # just resets the index of the df

In [21]:
ranking_data = {i:[-1 for _ in range(len(D2_voters_df))] for i in range(1,7)}

for voter_index, row in D2_voters_df.iterrows():
    for rank_position in range(1,7):
        num_votes_cast = row[rank_columns[rank_position]].sum()

        if num_votes_cast == 0:
            cast_vote = ""

        elif num_votes_cast > 1:
            cast_vote = "overvote"

            # here we lost knowledge of who was in the overvote. That's how Portland runs their election
            # system, but it could be interesting to study who is in the overvote!

        else:
            # find candidate name from column
            pd_series = row[rank_columns[rank_position]]
            cast_vote_column_name = pd_series.loc[pd_series == 1].index.tolist()[0]
            cast_vote = cast_vote_column_name.split(":")[-2]

        ranking_data[rank_position][voter_index] = cast_vote

# add the new columns
for rank_position in range(1,7):
    D2_voters_df[f"Rank {rank_position}"] = ranking_data[rank_position]

In [22]:
D2_voters_df.head()

,RowNumber,BoxID,BoxPosition,BallotID,PrecinctID,BallotStyleID,PrecinctStyleName,ScanComputerName,Status,Remade,...,"Choice_50004_1:City of Portland, Councilor, District 2:3:Number of Winners 3:Uncertified Write In:NON","Choice_50004_1:City of Portland, Councilor, District 2:4:Number of Winners 3:Uncertified Write In:NON","Choice_50004_1:City of Portland, Councilor, District 2:5:Number of Winners 3:Uncertified Write In:NON","Choice_50004_1:City of Portland, Councilor, District 2:6:Number of Winners 3:Uncertified Write In:NON",Rank 1,Rank 2,Rank 3,Rank 4,Rank 5,Rank 6
0,9,RCV-0001,9,RCV-0001+10019,73,4,4305-1,ScanStation6,0,0,...,0,0,0,0,Marnie Glickman,Elana Pirtle-Guiney,Jonathan Tasini,Dan Ryan,,
1,10,RCV-0001,10,RCV-0001+10021,73,4,4305-1,ScanStation6,0,0,...,0,0,0,0,Marnie Glickman,Dan Ryan,Nabil Zaghloul,,,
2,11,RCV-0001,11,RCV-0001+10023,69,4,3308-1,ScanStation6,0,0,...,0,0,0,0,Sameer Kanal,Michelle DePass,Tiffani Penson,Nat West,,
3,12,RCV-0001,12,RCV-0001+10025,2,4,4501-1,ScanStation6,0,0,...,0,0,0,0,Dan Ryan,Tiffani Penson,Nat West,Mariah Hudson,Jonathan Tasini,Elana Pirtle-Guiney
4,36,RCV-0001,36,RCV-0001+10073,28,4,4304-1,ScanStation6,0,0,...,0,0,0,0,Jennifer Park,Liz Taylor,Bob Simril,Laura Streib,Michelle DePass,Debbie Kitchin


In [ ]:
D2_voters_df[[f"Rank {rank_position}" for rank_position in range(1,7)]].to_csv("Portland_D2_raw_votekit_format.csv")

In [23]:
from votekit.cvr_loaders import load_csv

profile_D2 = load_csv("Portland_D2_raw_votekit_format.csv", rank_cols=[1,2,3,4,5,6])

In [24]:
profile_D2.candidates

('Antonio Jamal PettyJohnBlue',
 'Bob Simril',
 'Chris Olson',
 'Michelle DePass',
 'Sameer Kanal',
 'Debbie Kitchin',
 'Laura Streib',
 'Dan Ryan',
 'Will Mespelt',
 'James Armstrong',
 'Mariah Hudson',
 'Marnie Glickman',
 'Tiffani Penson',
 'Nabil Zaghloul',
 'Elana Pirtle-Guiney',
 'Liz Taylor',
 'Jennifer Park',
 'Nat West',
 'overvote',
 'Reuben Berlin',
 'Michael (Mike) Marshall',
 'Sam Sachs',
 'Write-in-123',
 'Jonathan Tasini',
 'Uncertified Write In',
 'Write-in-124',
 'Write-in-125')

In [25]:
from votekit.cleaning import remove_and_condense

profile_portland_D2 = remove_and_condense(['overvote','Write-in-123','Uncertified Write In','Write-in-124', 'Write-in-125'], profile_D2)

In [26]:
# code for computing the σIIA scores for ranking version

# similar code could be used for computing the σIIA scores for the winner set version 
#chanhge the directory to import the fairness metrics and voting rules if needed

import os
os.chdir('/Users/ss2776/Desktop/Journal_QRAA/Quantitative_fairness') 

# example with  Plurality

from Voting_rules import Ranked_Plurality

from fairness_metric import sigma_IIA

sigma_IIA(profile_portland_D2, Ranked_Plurality)

0.9939393939393939

In [27]:
# code for computing the σUM scores for winner set version

# similar code could be used for computing the σIIA scores for the ranking set version 
#chanhge the directory to import the fairness metrics and voting rules if needed
 

# example with  Borda
from votekit.elections import Borda
from fairness_metric import sigma_UM_winner_set

sigma_UM_winner_set(profile_portland_D2,Borda,3)

1.0